In [ ]:
#!pip install vk_api
#!pip install requests
#!pip install pandas

In [1]:
import vk_api
import requests
import pandas as pd
import time
import re
import json
from io import StringIO

1. Зайти на https://vk.com/apps?act=manage и нажать "Создать приложение"
2. Ввести любое название и выбрать "Standalone-приложение" в пункте "платформа"
3. Нажать "Подключить приложение"
4. На открывшейся странице нажать "Настройки" и скопировать "ID приложения"
5. Вставить его ниже в CLIENT_ID

In [2]:
CLIENT_ID = '7481982' #ID приложения

6. Запустить строчку ниже и перейти по ссылке
7. Скопировать из адресной строки код после 'acces_token'

In [141]:
link = 'https://oauth.vk.com/authorize?client_id=' + CLIENT_ID + '&scope=friends&response_type=token&v=5.103'
link

'https://oauth.vk.com/authorize?client_id=7481982&scope=friends&response_type=token&v=5.103'

8. Вставить код в access_token в строчку ниже

In [142]:
access_token = 'd8cfd64dffb0dbf619439d02834a4f3fb84130dc19031ba93dec46e22a6af5e4b586cef4aaa8c702a4ec9' 

9. Код ниже можно запускать подряд. Он должен отработать примерно за полчаса (или за полтора, если появится ошибка 'Error occure more than 10 times in a row, will be sleeping for 40 minutes')

In [143]:
base_url = "https://api.vk.com/method/%s"

In [216]:
school_n = 420

In [217]:
file_name = 'schoolers_final' + str(school_n) + '.json'

with open(file_name) as f:
    text = f.read()
    one_school = json.loads(text)

In [218]:
birth_year = [i for i in range(1996, 2007)]
sexes = [1, 2]
search_params = []
for year in birth_year:
    for sex in sexes:
        search_params.append((year, sex))

In [219]:
persons_total = 0
for sp in search_params:
    year = str(sp[0])
    sex = str(sp[1])
    persons_total += len(one_school[year][sex])
persons_total

1470

In [220]:
def call(url, params):
    return requests.get(url, params=params)

In [221]:
def wall_get(owner_id):
        params = {
            'v': '5.103',
            'owner_id': owner_id,
            'count': 1000
        }
        params = dict(params, **{
            "access_token": access_token
        })
        #print(params)
        #print(base_url % "wall.get", params)
        return call(base_url % "wall.get", params)

In [222]:
def make_response_dict(response):
    response_dict = []
    for item in response['response']['items']:
        if item['owner_id']==item['from_id']:
            attachments = []
            if 'attachments' in item:
                attachments = [a['type'] for a in item['attachments']]
            response_dict.append({
                'date': item['date'],
                'text': item['text'],
                'attachments': attachments
            })
    return response_dict

In [223]:
birth_year = [i for i in range(1996, 2007)]
sexes = [1, 2]
search_params = []
for year in birth_year:
    for sex in sexes:
        search_params.append((year, sex))
search_params[0]

(1996, 1)

In [224]:
persons_total = 0
for sp in search_params:
    year = str(sp[0])
    sex = str(sp[1])
    persons_total += len(one_school[year][sex])
persons_total

1470

In [225]:
def get_post(person_id):
    error_counter = 0
    response = wall_get(person_id).json()
    while 'error' in response:
        if response['error']['error_code'] == 30:
            return 'success', None, 0
        else:
            while 'error' in response:
                #print(response)
                safe_info = {'person_id': person_id}
                if response['error']['error_code'] == 6:
                    if error_counter < 3:
                        time.sleep(3) 
                        error_counter += 1 
                        response = wall_get(person_id).json()
                    elif 3 <= error_counter < 10:
                        time.sleep(60) 
                        error_counter += 1 
                        response = wall_get(person_id).json()
                    elif error_counter == 10:
                        print('Error occure more than 10 times in a row, will be sleeping for 40 minutes')
                        time.sleep(2400)
                        error_counter +=1
                        response = wall_get(person_id).json()
                    elif error_counter > 10:
                        return 'fail', 'Too much errors occured, safe point was made, try in 40 minutes', safe_info
                elif response['error']['error_code'] == 29:
                    message = 'Rate limit reached, try again tomorrow'
                    return 'fail', message, safe_info
                elif response['error']['error_code'] == 30:
                    return 'success', None, 0
                else:
                    return 'fail', response, 0
    return 'success', make_response_dict(response), 0

In [226]:
all_posts = {}
all_safe_info = []
done = 0
for sp in search_params:
    year = str(sp[0])
    sex = str(sp[1])
    for person_id in one_school[year][sex]:
        done += 1
        if done % 50 == 0:
            print(f'{done} out of {persons_total}: {done*100/persons_total} % done')
        status, answer, safe_info = get_post(person_id)
        if status == 'fail':
            print(answer)
            file_name2 = 'all_posts' + str(school_n) + '.json'
            with open(file_name2, "w") as write_file:
                json.dump(answer, write_file)
        if safe_info != 0:
            safe_info['param_ind'] = search_params.index(sp)
            file_name = 'start_file_' + str(school_n)
            with open('start_file_', "w") as write_file:
                json.dump(safe_info, write_file)
            print(safe_info)
        if status == 'success':
            all_posts[person_id] = answer

50 out of 1470: 3.401360544217687 % done
100 out of 1470: 6.802721088435374 % done
150 out of 1470: 10.204081632653061 % done
200 out of 1470: 13.605442176870747 % done
250 out of 1470: 17.006802721088434 % done
300 out of 1470: 20.408163265306122 % done
350 out of 1470: 23.80952380952381 % done
400 out of 1470: 27.210884353741495 % done
450 out of 1470: 30.612244897959183 % done
500 out of 1470: 34.01360544217687 % done
550 out of 1470: 37.414965986394556 % done
600 out of 1470: 40.816326530612244 % done
650 out of 1470: 44.21768707482993 % done
700 out of 1470: 47.61904761904762 % done
750 out of 1470: 51.02040816326531 % done
Rate limit reached, try again tomorrow
{'person_id': 266636940, 'param_ind': 9}
Rate limit reached, try again tomorrow
{'person_id': 257182030, 'param_ind': 9}
Rate limit reached, try again tomorrow
{'person_id': 243700888, 'param_ind': 9}
Rate limit reached, try again tomorrow
{'person_id': 238733584, 'param_ind': 9}
Rate limit reached, try again tomorrow
{'pe

Rate limit reached, try again tomorrow
{'person_id': 522042067, 'param_ind': 9}
Rate limit reached, try again tomorrow
{'person_id': 518229208, 'param_ind': 9}
Rate limit reached, try again tomorrow
{'person_id': 453459296, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 203670003, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 206869876, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 486876614, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 487091302, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 253046347, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 202093387, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 221460727, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 218889453, 'param_ind': 10}
Rate limit reached, try again tomorrow
{'person_id': 457818177, 'param_ind': 10}
Rate limit reached, try again 

Rate limit reached, try again tomorrow
{'person_id': 133987680, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 184295531, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 139149325, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 454638895, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 446078272, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 430358997, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 294430611, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 239676280, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 124074528, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 152689568, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 361992075, 'param_ind': 11}
Rate limit reached, try again tomorrow
{'person_id': 200097294, 'param_ind': 11}
Rate limit reached, try agai

Rate limit reached, try again tomorrow
{'person_id': 41941140, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 513924857, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 471526449, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 447198384, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 389568102, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 381330307, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 223647868, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 512849636, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 346896431, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 324965474, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 358769470, 'param_ind': 13}
Rate limit reached, try again tomorrow
{'person_id': 302704390, 'param_ind': 13}
Rate limit reached, try again

Rate limit reached, try again tomorrow
{'person_id': 207675109, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 278627959, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 400857286, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 202078113, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 504740254, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 465358319, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 226295916, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 480828526, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 457368699, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 476237619, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 510752285, 'param_ind': 15}
Rate limit reached, try again tomorrow
{'person_id': 487298999, 'param_ind': 15}
Rate limit reached, try agai

Rate limit reached, try again tomorrow
{'person_id': 481838283, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 476241388, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 425882368, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 384723938, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 486055674, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 475236756, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 481027112, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 510783645, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 503409599, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 488174750, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 384906900, 'param_ind': 17}
Rate limit reached, try again tomorrow
{'person_id': 482742305, 'param_ind': 17}
Rate limit reached, try agai

Rate limit reached, try again tomorrow
{'person_id': 541971354, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 541152000, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 538338269, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 537978126, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 537714386, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 537496787, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 536133312, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 533631520, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 532754201, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 529906648, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 529684543, 'param_ind': 18}
Rate limit reached, try again tomorrow
{'person_id': 529227488, 'param_ind': 18}
Rate limit reached, try agai

Rate limit reached, try again tomorrow
{'person_id': 582262070, 'param_ind': 21}
Rate limit reached, try again tomorrow
{'person_id': 581447286, 'param_ind': 21}
Rate limit reached, try again tomorrow
{'person_id': 577835780, 'param_ind': 21}
Rate limit reached, try again tomorrow
{'person_id': 577169084, 'param_ind': 21}
Rate limit reached, try again tomorrow
{'person_id': 539364937, 'param_ind': 21}
Rate limit reached, try again tomorrow
{'person_id': 532087411, 'param_ind': 21}
Rate limit reached, try again tomorrow
{'person_id': 516094919, 'param_ind': 21}


In [215]:
file_name2 = 'all_posts' + str(school_n) + '.json'
with open(file_name2, "w") as write_file:
    json.dump(all_posts, write_file)

In [227]:
with open('all_posts49293.json') as f:
    text = f.read()
    ap = json.loads(text)

In [229]:
ap.keys()

dict_keys(['13954704', '41806475', '6150509', '32890669', '18111501', '4215713', '313767329', '13062075', '4634922', '113518104', '13570319', '71487335', '20069671', '11104363', '66734327', '14486850', '13022479', '161929802', '157287594', '52442372', '20393769', '13460301', '12403684', '10755507', '8477565', '7797907', '5740615', '250158989', '74628668', '56517681', '49883518', '35110072', '15864232', '8172024', '266631232', '219368605', '174853076', '97677456', '18103886', '8852845', '99186356', '34657652', '379258514', '334084270', '284056855', '10323114', '137396371', '31289755', '318844875', '21286110', '14925601', '28727458', '11084656', '74188840', '298021463', '274680333', '232594036', '255674275', '54703647', '32191879', '11089279', '101545863', '23187870', '20770032', '20389370', '215423490', '23891357', '303702713', '126518256', '123713884', '57419540', '21768669', '29068795', '123411842', '99320217', '54927529', '34945045', '28889149', '24355618', '22932040', '13573531', '4